In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("Telecom_Data.csv")
data.head()

,Age,Married,Number of Dependents,Tenure in Months,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Avg Monthly GB Download,Online Security,...,Contract_One Year,Contract_Two Year,Internet Type_Cable,Internet Type_DSL,Internet Type_Fiber Optic,Offer_Offer A,Offer_Offer B,Offer_Offer C,Offer_Offer D,Offer_Offer E
0,37,1,0,9,1,42.39,0,1,16.0,0,...,1,0,1,0,0,0,0,0,0,0
1,46,0,0,9,1,10.69,1,1,10.0,0,...,0,0,1,0,0,0,0,0,0,0
2,50,0,0,4,1,33.65,0,1,30.0,0,...,0,0,0,0,1,0,0,0,0,1
3,78,1,0,13,1,27.82,0,1,4.0,0,...,0,0,0,0,1,0,0,0,1,0
4,75,1,0,3,1,7.38,0,1,11.0,0,...,0,0,0,0,1,0,0,0,0,0


In [3]:
from sklearn.model_selection import GridSearchCV,cross_val_score,RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings('ignore')

In [4]:
data.shape

(5000, 38)

In [5]:
(data['Customer Status_Stayed']).value_counts()

1    3647
0    1353
Name: Customer Status_Stayed, dtype: int64

In [6]:
X= data.drop("Customer Status_Stayed",1)
y = data['Customer Status_Stayed']

# Should I Scale my data before using Cross_validation_score?

### This notebook shows the difference between the Scaled data and the actual data on cross_val_score

##### What is the purpose of cross-validation in machine learning?
Cross-validation is a technique for evaluating ML models by training several ML models on subsets of the available input data and evaluating them on the complementary subset of the data.

# Actual Data

In [7]:
log = (cross_val_score(LogisticRegression(max_iter =1000),X,y,cv =5)).mean()

In [8]:
svc = cross_val_score(SVC(),X,y,cv =5).mean()

In [9]:
#SVC().get_key()

In [10]:
dtc = cross_val_score(DecisionTreeClassifier(),X,y,cv =5).mean()

In [11]:
rfc = cross_val_score(RandomForestClassifier(n_estimators = 10),X,y,cv =5).mean()

In [12]:
knn = cross_val_score(KNeighborsClassifier(),X,y,cv =5).mean()

In [13]:
scale = MinMaxScaler()
X_scale = scale.fit_transform(X)

# Scaled data

In [14]:
logScale =(cross_val_score(LogisticRegression(max_iter =1000),X_scale,y,cv =5)).mean()

In [15]:
svc_scale = cross_val_score(SVC(),X_scale,y,cv =5).mean()

In [16]:
knn_scale = cross_val_score(KNeighborsClassifier(),X_scale,y,cv =5).mean()

In [17]:
rfc_scale = cross_val_score(RandomForestClassifier(n_estimators = 10),X_scale,y,cv =5).mean()

In [18]:
dtc_scale = cross_val_score(DecisionTreeClassifier(),X,y,cv =5).mean()

In [19]:
score = log,svc,dtc,rfc,knn
score2 =logScale,svc_scale,dtc_scale,rfc_scale,knn_scale

In [20]:
model = { 'data': score, 'Scale_data': score2, 'name':['LogisticRegression','SVC','DecisionTree','RandomFores','KNN']}
pd.DataFrame(model).set_index('name')

,data,Scale_data
name,,
LogisticRegression,0.8148,0.8206
SVC,0.7294,0.8160
DecisionTree,0.7474,0.7522
RandomFores,0.7950,0.7908
KNN,0.7448,0.7722


# Conclusion 

### As shown above Scaling the data increased the accuracy of some model a little bit.
### scaling before applying cross_val_score is not that necessary.

# LOGISTIC REGRESSION

In [21]:
X_train,X_test,y_train,y_test = train_test_split(X_scale,y,test_size =0.3,random_state =0)

In [22]:
logmodel = LogisticRegression()
logmodel = logmodel.fit(X_train,y_train)
acc =logmodel.score(X_test,y_test)
print('Accuracy Score = ', acc)

Accuracy Score =  0.8253333333333334


# PCA.   Principal Component Analysis

Principal Component Analysis (PCA) is an unsupervised,statistical technique primarily used for dimensionality reduction in machine learning.
#### Major Advantage     
dimensionality reduction, information compression, data de-noising

### cons
it sometimes reduces the accuracy of a model when all features contribute to the outcome

In [23]:
from sklearn.decomposition import PCA

pca = PCA(25)
X_pca =pca.fit_transform(X)
X_pca

array([[-2.68807308e+03,  1.07510571e+02,  2.69057322e+00, ...,
         3.97228572e-01, -2.02082579e-01, -4.69897539e-02],
       [-3.05093521e+03, -1.83886976e+02, -2.23024049e+01, ...,
         2.28564315e-01, -6.99101492e-01, -1.50429266e-01],
       [-3.35396062e+03, -6.01829115e+01,  7.98955834e+00, ...,
        -4.02975331e-01, -1.08034759e-01,  2.91507601e-01],
       ...,
       [-3.59697615e+03, -1.29124727e+02,  1.09690549e+01, ...,
         4.13343825e-01,  1.26956441e-01, -3.61468489e-02],
       [ 3.04441784e+03,  7.30513809e+02, -1.72173585e+00, ...,
        -9.16237486e-02, -1.95324697e-02,  6.36029843e-01],
       [ 5.97122917e+02, -2.00023025e+02, -1.45300210e+01, ...,
        -4.50066677e-01, -1.19769789e-01, -1.01770195e-02]])

In [24]:
X_pca.shape

(5000, 25)

### Using PCA I have reduced the features to 20.

In [25]:
pca.explained_variance_ratio_

array([9.63722883e-01, 3.61066025e-02, 5.91811074e-05, 4.98775623e-05,
       3.17952384e-05, 1.14819509e-05, 8.47739254e-06, 5.01266965e-06,
       4.38727366e-06, 6.30453086e-08, 2.66311155e-08, 2.00484562e-08,
       1.77347149e-08, 1.71476627e-08, 1.60425428e-08, 1.43885808e-08,
       1.31639372e-08, 1.21206088e-08, 1.10780813e-08, 1.06446825e-08,
       1.01443548e-08, 9.55796260e-09, 8.68569275e-09, 8.03995019e-09,
       6.97598777e-09])

## using the pca features to build a model

In [26]:
X_train,X_test,y_train,y_test = train_test_split(X_pca,y,test_size =0.3,random_state =0)

In [27]:
model = LogisticRegression()
model = model.fit(X_train,y_train)
acc = model.score(X_test,y_test)
print(' PCA Accuracy Score = ', acc)

 PCA Accuracy Score =  0.776


# Conclusion 

After applying PCA the dimention of the data reduces and the computation was fast but it leads to reduction in accuracy value